# Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
import pytz
import datetime 
import astral
import astropy.time as aptime
import os
import glob

# Read in text files

In [ ]:
#txt_file1 = pd.read_csv('IFS_AllOnSky_RawDistorcorr_aotelem_CleanSee30_20190226.txt')
txt_file1= pd.read_csv('IFS_AllOnSky_RawDistorcorr_aotelem_CleanSee30_20190227.txt')
txt_file1_copy = copy.copy(txt_file1)
raw_contrast_data= pd.DataFrame(txt_file1_copy)
print len(raw_contrast_data)

raw_contrast_data['dts'] = aptime.Time(raw_contrast_data['MJDOBS'], format='mjd').datetime 
raw_contrast_data['dts'] = raw_contrast_data['dts'].dt.tz_localize(pytz.UTC)

In [ ]:
raw_contrast_data.head()

In [ ]:
txt_file2 = pd.read_csv('AO_telem_20180207.txt')
txt_file2_copy = copy.copy(txt_file2)
ao_telem_data = pd.DataFrame(txt_file2_copy)
print len(ao_telem_data)

ao_telem_data['dts'] = pd.to_datetime(ao_telem_data['whenstr'], utc=True, format = '%Y%m%d%H%M%S')
ao_telem_data['dts'] = ao_telem_data['dts'].dt.tz_convert(pytz.UTC)
ao_telem_data['whenstr'] = ao_telem_data['whenstr'].astype('str')

#phase filename
ao_telem_data['phase_file'] = 'When_' 
ao_telem_data['phase_file'] = ao_telem_data['phase_file'] + ao_telem_data.loc[:,'dts'].dt.strftime('%Y.%-m.%-d_%-H.%-M.%-S')

#phase directory 
ao_telem_data['phase_dir'] = ao_telem_data.loc[:,'whenstr'].str[0:8]

# Merge IFS data with AO telemetry

Fill missing IFS data with nan

In [ ]:
ao_telem_raw_IFS_data =  pd.merge_asof(ao_telem_data, raw_contrast_data, by='OBJNAME',on='dts',tolerance=pd.Timedelta('1min'))

In [ ]:
print len(ao_telem_raw_IFS_data)
print sum(np.isfinite(ao_telem_raw_IFS_data['MASSTAU']))
print sum(np.isfinite(ao_telem_raw_IFS_data['WFE']))

# Output into a list of files

In [ ]:
phase_samples = raw_contrast_data.loc[:,'phase_file']
phase_samples = phase_samples.drop_duplicates(keep='first').values
phase_samples = pd.DataFrame({'phase_file':list(phase_samples)})

In [ ]:
save_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/datatables/'

In [ ]:
phase_samples.to_csv(save_path+'phase_samples_20180226.txt',index=False,header=False)

# Make a list that leaves out filenames already analyzed 

In [ ]:
new_phase_samples = raw_contrast_data.loc[:,'phase_file']
new_phase_samples = new_phase_samples.drop_duplicates(keep='first').values

In [ ]:
old_phase_samples = pd.read_csv('phase_samples_20181105.txt',header=None)
old_phase_samples = pd.Series(old_phase_samples[0]).values

In [ ]:
new_phase_samples.update(old_phase_samples)

new = set(new_phase_samples)
old = set(old_phase_samples) 

In [ ]:
phase_samples = pd.DataFrame({'phase_file':list(new_phase_samples)})

# Reproduce IFS sample with AO telem data

Fill missing ao telem data with nans

In [ ]:
def filter_data(data, imag = 10., tau = 1., see = 2., all_cond = True):
    "This selects all rows of input dataframe that satisfy conditions set by the user"
    
    date1 = datetime.date(year = 2014, month = 1, day =1)
    
    #  AO system requirements
    cond1 = data['dts'] > date1 
    cond2 = data['COADDS'] == 1
    cond3 = (data['OBSMODE'] == 'H_coron')|(data['OBSMODE'] == 'Spec')
    #cond4 = data['AOFRAMES'] == 1000

    #  Good seeing conditions
    cond5 = data['IMAG'] < imag #Bright Stars
    cond6 = data['MASSTAU'] > tau  # slow moving turbulence [ms]
    cond7 = data['DIMMSEE'] < see  # smaller scale turbulence ["]
    
    if all_cond:
        ind = np.where(cond1 & cond2 & cond3 & cond5 & cond6 & cond7)[0]  
        print len(ind)
    else:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5)[0]  
        print len(ind)

    filtered_data = data.iloc[ind]
    #filtered_data = filtered_data.dropna(subset = ['CONTR040','cal_wfe','M1_avg','TAMBIENT','Outside_OE_temperature'])
    
    new_ind = np.arange(len(filtered_data))
    filtered_data = filtered_data.set_index(new_ind)
        
    return filtered_data

In [ ]:
r = filter_data(raw_contrast_data,all_cond=True)

drop rows missing mass tau measurements and wfe measurements

In [ ]:
raw_IFS_ao_telem_data =  pd.merge_asof(r, ao_telem_data, on='dts',tolerance=pd.Timedelta('1min')).dropna(subset=['WFE']).dropna(subset=['MASSTAU']).dropna(subset=['DIMMSEE'])

# Instances of best and worst seeing

In [ ]:
ao_telem_raw_IFS_data = ao_telem_raw_IFS_data.dropna(subset=['phase_file','M1_avg','TAMBIENT'])

In [ ]:
len(ao_telem_raw_IFS_data)

In [ ]:
def search_for_dome_seeing(data, sample_size, presence = True):
    
    cut = sample_size
    sample = set()
    
    if presence:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.M1_avg - data.T_twr).sort_values(ascending = False)[0:cut]
            contrast = data.CONTR040.sort_values(ascending = False)[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index)
            cut = cut + 1
            
    else:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.M1_avg - data.T_twr).sort_values()[0:cut]
            contrast = data.CONTR040.sort_values()[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index)
            cut = cut + 1
    return list(sample)  

In [ ]:
def search_for_dome_seeing(data, sample_size, presence = True):
    
    cut = sample_size
    sample = set()
    
    if presence:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.OMSATEMP - data.TAMBIENT).sort_values(ascending = False)[0:cut]
            contrast = data.CONTR040.sort_values(ascending = False)[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index)
            cut = cut + 1
            
    else:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.OMSATEMP - data.TAMBIENT).sort_values()[0:cut]
            contrast = data.CONTR040.sort_values()[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index)
            cut = cut + 1
    return list(sample)  

In [ ]:
sample = r.loc[search_for_dome_seeing(r,50,presence=False),
                       ['month','OBJNAME','dts','phase_file','CONTR040','OMSATEMP','TAMBIENT','MASSTAU','IMAG']].sort_values("CONTR040",ascending = False)

# Grouping

In [ ]:
g = sample.groupby(['month','OBJNAME'])

In [ ]:
no_dome_seeing_list = set()
for name,group in g:
    print(name)
    print(group['phase_file'].values,group['OMSATEMP'].values,group['TAMBIENT'].values)
    no_dome_seeing_list.update(group['OBJNAME'].values)

In [ ]:
sample_list = pd.DataFrame({'phase_file':list(sample_list)})

In [ ]:
sample_list.to_csv(save_path+'clean_seeing_phase_samples_20190205.txt',index=False,header=False)

# Debugging

In [ ]:
dome_seeing_list

In [ ]:
no_dome_seeing_list

In [ ]:
dome_seeing_list.intersection(no_dome_seeing_list)

In [ ]:
c_eri = r.iloc[np.where(r['OBJNAME']=='c Eri')]

In [ ]:
%matplotlib notebook

plt.figure(1)
plt.plot(c_eri['OMSATEMP']-c_eri['TAMBIENT'],c_eri['CONTR040'],'.')
plt.yscale('log')

In [ ]:
aotelem = pd.read_csv('aotelem_20190226.txt')

In [ ]:
Eri_list =list(aotelem.loc[np.where(aotelem['objname']=='c Eri')[0],'datafile'])

In [ ]:
result = [re.search('ugp_(.*)_phase', n).group(1) for n in Eri_list]

In [ ]:
result = set(result)

In [ ]:
sample_list = pd.DataFrame({'phase_file':list(result)})
sample_list.to_csv(save_path+'c_Eri_samples.txt',index=False,header=False)

In [ ]:
l1 = set(pd.read_csv(save_path+'phase_samples_20181116.txt',header=None)[0].values)
l2 = set(pd.read_csv(save_path+'phase_samples_20181116.txt',header=None)[0].values)
l3 = set(pd.read_csv(save_path+'phase_samples_20181116.txt',header=None)[0].values)

In [ ]:
l123=set()
l123.update(l1)
l123.update(l2)
l123.update(l3)

In [ ]:
l123.intersection_update(set(result))

In [ ]:
s = result &= l123